In [1]:
print("Hello")
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("GPU is available!")
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("GPU is not available.")

Hello
GPU is available!
GPU Name: NVIDIA GeForce RTX 4090


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
# !pip install -r requirements.txt

# !pip install -q -U torch --index-url https://download.pytorch.org/whl/cu117
# !pip install -q -U transformers=="4.39.1"
# !pip install -q accelerate=="0.28.0"
# !pip install -q -i https://pypi.org/simple/ bitsandbytes=="0.43.0"
# !pip install -q -U datasets=="2.17.0"
# !pip install -q -U git+https://github.com/huggingface/trl
# !pip install -q -U git+https://github.com/huggingface/peft

# !pip install torch==2.1.2 tensorboard
# !pip install --upgrade transformers==4.38.2
# !pip install datasets==2.16.1
# !pip install accelerate==0.26.1
# !pip install evaluate==0.4.1
# !pip install bitsandbytes==0.42.0
# !pip install trl==0.7.11
# !pip install peft==0.8.2

In [4]:
# !pip install transformers==4.40


In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
import bitsandbytes as bnb

from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split

In [7]:
print(f"pytorch version {torch.__version__}")

pytorch version 2.3.1+cu121


# 1 Load Dataset

In [8]:
filename = "./data/all-data.csv"

df = pd.read_csv(filename, 
                 names=["sentiment", "text"],
                 encoding="utf-8", encoding_errors="replace")
df.shape

(4846, 2)

In [9]:
df.sentiment.value_counts()

sentiment
neutral     2879
positive    1363
negative     604
Name: count, dtype: int64

In [10]:
pd.set_option('display.max_colwidth', 200)
df.head()

,sentiment,text
0,neutral,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing ."
1,neutral,"Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said ."
2,negative,The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers ...
3,positive,With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profi...
4,positive,"According to the company 's updated strategy for the years 2009-2012 , Basware targets a long-term net sales growth in the range of 20 % -40 % with an operating profit margin of 10 % -20 % of net ..."


# 2 Split into training, test and eval data set

In [11]:
from main.feature_engineer import DataFrameSplitter


splitter = DataFrameSplitter(df, train_size=300)

# Perform the split to obtain train, test, and eval sets
X_train_df, X_test_df, X_eval_df = splitter.split()
X_test_df.to_csv("data/X_test_df.csv", index=False)


# Print shapes and target column breakup of the train data
print(f'Shape of train, test, and eval data are {X_train_df.shape}, {X_test_df.shape}, {X_eval_df.shape}')
print(f'\nBreak up by target column in train data is\n{X_train_df.sentiment.value_counts()}')
    

Shape of train, test, and eval data are (900, 2), (900, 2), (150, 2)

Break up by target column in train data is
sentiment
neutral     300
positive    300
negative    300
Name: count, dtype: int64


# 3 Prompt Engineering

In [12]:
from main.prompt_engineer import PromptGenerator

# Assuming 'X_train_df', 'X_eval_df', and 'X_test_df' are your DataFrames containing text (and sentiment for 'X_train_df' and 'X_eval_df')
prompt_generator = PromptGenerator()

# Generate training and validation prompts
X_train_df = prompt_generator.generate_dataframe_prompts(X_train_df, prompt_type='train')
X_eval_df = prompt_generator.generate_dataframe_prompts(X_eval_df, prompt_type='train')




# Generate test prompts
y_true = X_test_df.sentiment
X_test_df = prompt_generator.generate_dataframe_prompts(X_test_df, prompt_type='test')


In [13]:
#convert pandas dataframe to Huggingface dataset
train_data = Dataset.from_pandas(X_train_df)
eval_data = Dataset.from_pandas(X_eval_df)

# 4 Load Base Model

In [14]:
from main.model_base import QuantizedBaseModelInitializer

# model_name = "MaziyarPanahi/Mistral-7B-Instruct-v0.2"
#model_name = "NousResearch/Llama-2-7b-hf"'
# model_name = "mhenrichsen/gemma-7b"
# model_name = "microsoft/phi-2"
# model_name = "microsoft/Phi-3-mini-4k-instruct"
model_name = "NousResearch/Meta-Llama-3-8B-Instruct"
# Create an instance of QuantizedBaseModelInitializer
initializer = QuantizedBaseModelInitializer(model_name)

# Initialize the model and tokenizer with quantization
model, tokenizer = initializer.initialize()

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# 5 Evaluate Base Model Performance

In [15]:
from main.predict import ModelPredictor
from main.evaluation import ModelEvaluator



predictor = ModelPredictor(model, tokenizer)
y_pred = predictor.predict(X_test_df)


evaluator = ModelEvaluator()
evaluator.evaluate(y_true, y_pred)

Predicting:   0%|          | 0/900 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Accuracy: 0.332
Accuracy for label 0: 0.000
Accuracy for label 1: 0.973
Accuracy for label 2: 0.023

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       300
           1       0.33      0.97      0.49       300
           2       0.47      0.02      0.04       300

    accuracy                           0.33       900
   macro avg       0.27      0.33      0.18       900
weighted avg       0.27      0.33      0.18       900


Confusion Matrix:
[[  0 300   0]
 [  0 292   8]
 [  0 293   7]]


# 7 Fine-tune base model on given dataset

In [16]:
from main.train_with_fine_tuning import PEFTModelTrainer
trainer = PEFTModelTrainer(model, tokenizer, train_data, eval_data)

# Start the training process
trainer.train_model()

TypeError: PEFTModelTrainer.set_seed() takes from 0 to 1 positional arguments but 2 were given

# 8 Evaluate Fine-Tune Model Performance

In [ ]:

predictor = ModelPredictor(model, tokenizer)
y_pred = predictor.predict(X_test_df)


evaluator = ModelEvaluator()
evaluator.evaluate(y_true, y_pred)

Predicting:   0%|          | 0/900 [00:00<?, ?it/s]

Accuracy: 0.872
Accuracy for label 0: 0.933
Accuracy for label 1: 0.873
Accuracy for label 2: 0.810

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.93      0.95       300
           1       0.79      0.87      0.83       300
           2       0.88      0.81      0.85       300

    accuracy                           0.87       900
   macro avg       0.88      0.87      0.87       900
weighted avg       0.88      0.87      0.87       900


Confusion Matrix:
[[280  17   3]
 [  9 262  29]
 [  3  54 243]]


# 9 Predict with Fine-Tune Model